<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C1/W4/ungraded_labs/C1_W4_Lab_1_image_generator_no_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Training with ImageDataGenerator

In this lab, you will build a train a model on the [Horses or Humans](https://www.tensorflow.org/datasets/catalog/horses_or_humans) dataset. This contains over a thousand images of horses and humans with varying poses and filesizes. You will use the [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) class to prepare this dataset so it can be fed to a convolutional neural network.

**IMPORTANT NOTE:** This notebook is designed to run as a Colab. Running it on your local machine might result in some of the code blocks throwing errors.

Run the code below to download the compressed dataset `horse-or-human.zip`.

In [1]:
#!pip install keras

In [25]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from numpy.random import RandomState
from keras import backend as K
import tensorflow as tf
import numpy as np
from tensorflow import keras
import csv

In [26]:
dataframe = pd.read_csv('DR_BH_data_merged_for_viet.csv')

In [27]:
dataframe.head()

,Screening_No,Age,Height,Weight,WC,BMI,SBP,DBP,Glucose,HbA1c,Sex,DR_diagnosis_status
0,D-001,66,157.8,70.7,97.2,28.39,130,84,171,7.5,M,noDR
1,D-002,51,171.9,65.1,89.2,22.03,140,85,130,6.0,M,noDR
2,D-003,46,174.6,60.1,81.1,19.71,122,75,242,11.2,M,DR
3,D-004,60,169.7,62.7,75.5,21.77,117,69,117,7.9,M,noDR
4,D-005,45,160.0,73.5,89.5,28.71,144,85,134,7.5,F,noDR


In [28]:
dataframe_new = dataframe.drop('Screening_No', axis = 1)

In [29]:
dataframe_new.head()

,Age,Height,Weight,WC,BMI,SBP,DBP,Glucose,HbA1c,Sex,DR_diagnosis_status
0,66,157.8,70.7,97.2,28.39,130,84,171,7.5,M,noDR
1,51,171.9,65.1,89.2,22.03,140,85,130,6.0,M,noDR
2,46,174.6,60.1,81.1,19.71,122,75,242,11.2,M,DR
3,60,169.7,62.7,75.5,21.77,117,69,117,7.9,M,noDR
4,45,160.0,73.5,89.5,28.71,144,85,134,7.5,F,noDR


In [30]:
dataframe_new.dtypes

Age                      int64
Height                 float64
Weight                 float64
WC                     float64
BMI                    float64
SBP                      int64
DBP                      int64
Glucose                  int64
HbA1c                  float64
Sex                     object
DR_diagnosis_status     object
dtype: object

In [31]:
dataframe_new

,Age,Height,Weight,WC,BMI,SBP,DBP,Glucose,HbA1c,Sex,DR_diagnosis_status
0,66,157.8,70.7,97.2,28.39,130,84,171,7.5,M,noDR
1,51,171.9,65.1,89.2,22.03,140,85,130,6.0,M,noDR
2,46,174.6,60.1,81.1,19.71,122,75,242,11.2,M,DR
3,60,169.7,62.7,75.5,21.77,117,69,117,7.9,M,noDR
4,45,160.0,73.5,89.5,28.71,144,85,134,7.5,F,noDR
...,...,...,...,...,...,...,...,...,...,...,...
257,69,152.9,72.3,88.0,30.93,139,85,130,7.7,F,noDR
258,49,150.9,55.4,79.0,24.33,124,76,236,8.9,F,DR
259,56,163.4,79.5,86.0,29.78,150,79,199,8.1,M,noDR
260,73,150.6,55.4,67.0,24.43,120,70,131,7.2,F,noDR


In [32]:
# replacing values
dataframe_new['Sex'].replace(['M', 'F'],[0, 1], inplace=True)

In [33]:
# replacing values
dataframe_new['DR_diagnosis_status'].replace(['noDR', 'DR'],[0, 1], inplace=True)

In [34]:
dataframe_new.head()

,Age,Height,Weight,WC,BMI,SBP,DBP,Glucose,HbA1c,Sex,DR_diagnosis_status
0,66,157.8,70.7,97.2,28.39,130,84,171,7.5,0,0
1,51,171.9,65.1,89.2,22.03,140,85,130,6.0,0,0
2,46,174.6,60.1,81.1,19.71,122,75,242,11.2,0,1
3,60,169.7,62.7,75.5,21.77,117,69,117,7.9,0,0
4,45,160.0,73.5,89.5,28.71,144,85,134,7.5,1,0


In [35]:
# Reshape and normalize training data
X = dataframe_new.drop('DR_diagnosis_status', axis =1).values

X_train = X / 255.0

y_train = dataframe_new.DR_diagnosis_status

In [290]:
#!pip install np_utils

In [13]:
# from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding

# y_train = to_categorical(y_train, num_classes = 2)

In [14]:
#X_train

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=2)

In [294]:
# from sklearn.model_selection import train_test_split
# # train_split= .8  # percentage of df to use for training
# # test_split = .05 # percentage of df to use for test
# # note valid_split=1- train_split = test_ split in this case .15
# train_df, dummy_df= train_test_split(X_train, y_train)
# # dummy_split=test_split/(1.0 - train_split)
# # test_df, valid_df=train_test_split(dummy_df, train_split=dummy_split) # note dummy_df is 20% of df

In [295]:
# # Reshape and normalize training data
# X = dataframe.drop('DR_diagnosis_status', axis =1).values
# X = dataframe.drop('Screening_No', axis =1).values
# #X_train = trainX / 255.0

# y_train = dataframe.DR_diagnosis_status

In [296]:
# from sklearn import preprocessing
# lb = preprocessing.LabelBinarizer()
# y_train = lb.fit_transform(y_train)
# #y_test = lb.fit_transform(y_test)

In [37]:
dataframe_new

,Age,Height,Weight,WC,BMI,SBP,DBP,Glucose,HbA1c,Sex,DR_diagnosis_status
0,66,157.8,70.7,97.2,28.39,130,84,171,7.5,0,0
1,51,171.9,65.1,89.2,22.03,140,85,130,6.0,0,0
2,46,174.6,60.1,81.1,19.71,122,75,242,11.2,0,1
3,60,169.7,62.7,75.5,21.77,117,69,117,7.9,0,0
4,45,160.0,73.5,89.5,28.71,144,85,134,7.5,1,0
...,...,...,...,...,...,...,...,...,...,...,...
257,69,152.9,72.3,88.0,30.93,139,85,130,7.7,1,0
258,49,150.9,55.4,79.0,24.33,124,76,236,8.9,1,1
259,56,163.4,79.5,86.0,29.78,150,79,199,8.1,0,0
260,73,150.6,55.4,67.0,24.43,120,70,131,7.2,1,0


In [345]:
#x_train = np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))
#x_test = np.reshape(x_test, (_test.shape[0],1,x_test.shape[1]))

In [376]:
#x_train = X_train.reshape(-1, 235, 1)

In [17]:
#x_train = X_train.astype(int)

In [18]:
#x_train = np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))

In [19]:
#X_train.shape[1]

10

In [396]:
# train_generator = train_datagen.flow_from_directory(directory='./data/',
#                                                         target_size=(150, 150),
#                                                         batch_size=10,
#                                                         class_mode='binary')

In [397]:
#x_train = np.reshape(X_train, (-1, 235, 10))

In [38]:
X_train.shape[1]

10

In [43]:
X_train

array([[0.23529412, 0.64980392, 0.20823529, ..., 0.41568627, 0.03019608,
        0.        ],
       [0.24313725, 0.68980392, 0.29686275, ..., 0.66666667, 0.02901961,
        0.        ],
       [0.16470588, 0.66784314, 0.29294118, ..., 0.25098039, 0.02352941,
        0.        ],
       ...,
       [0.28627451, 0.60392157, 0.22313725, ..., 0.67843137, 0.03098039,
        0.00392157],
       [0.31764706, 0.65137255, 0.26745098, ..., 0.6       , 0.03137255,
        0.00392157],
       [0.25098039, 0.57294118, 0.16705882, ..., 0.53333333, 0.02980392,
        0.00392157]])

In [44]:
x_train = np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))

In [45]:
x_train

array([[[0.23529412, 0.64980392, 0.20823529, ..., 0.41568627,
         0.03019608, 0.        ]],

       [[0.24313725, 0.68980392, 0.29686275, ..., 0.66666667,
         0.02901961, 0.        ]],

       [[0.16470588, 0.66784314, 0.29294118, ..., 0.25098039,
         0.02352941, 0.        ]],

       ...,

       [[0.28627451, 0.60392157, 0.22313725, ..., 0.67843137,
         0.03098039, 0.00392157]],

       [[0.31764706, 0.65137255, 0.26745098, ..., 0.6       ,
         0.03137255, 0.00392157]],

       [[0.25098039, 0.57294118, 0.16705882, ..., 0.53333333,
         0.02980392, 0.00392157]]])

In [53]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers, losses
from tensorflow.keras.optimizers import RMSprop

model = keras.models.Sequential()
#model.add(tf.keras.layers.InputLayer())
model.add(Conv1D(filters=6, kernel_size=21, strides=1, padding='valid', activation='relu',input_shape=(235, 10),kernel_initializer=keras.initializers.he_normal()))
model.add(BatchNormalization()) #what is the purpose of this!
model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
model.add(Conv1D(filters=16, kernel_size=5, strides=1, padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
# model.add(Dense(84))
# model.add(Dense(5, activation='softmax'))
# model.add(Dense(10))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 215, 6)            1266      
                                                                 
 batch_normalization_8 (Bat  (None, 215, 6)            24        
 chNormalization)                                                
                                                                 
 max_pooling1d_8 (MaxPoolin  (None, 107, 6)            0         
 g1D)                                                            
                                                                 
 conv1d_10 (Conv1D)          (None, 103, 16)           496       
                                                                 
 batch_normalization_9 (Bat  (None, 103, 16)           64        
 chNormalization)                                                
                                                      

In [54]:
model.compile(loss= 'binary_crossentropy' , optimizer= RMSprop(learning_rate=0.001), metrics=[ 'accuracy' ])

In [55]:
x_train.shape

(235, 1, 10)

In [49]:
#x_train = x_train.reshape(-1, 235, 10)

In [426]:
#x_train.size

2350

In [51]:
model.fit(x_train, Y_train, epochs = 50)

Epoch 1/50


ValueError: in user code:

    File "C:\Users\wlgus\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\wlgus\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\wlgus\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\wlgus\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\wlgus\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\wlgus\anaconda3\Lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

